In [0]:
%python
dbutils.fs.rm("dbfs:/user/hive/warehouse/team", True)

Out[1]: True

# UEFA EURO 2024 play-off draw

- Semi-finals – Path C: Georgia (1) vs Luxembourg (4), Greece (2) vs Kazakhstan (3)

- Semi-finals – Path B: Israel (1) vs Iceland (4), Bosnia and Herzegovina (2) vs Ukraine (3)

- Semi-finals – Path A: Poland (1) vs Estonia (4), Wales (2) vs Finland (3)

*************************************************************************************************************************************

#### Final Path C: Georgia / Luxembourg vs Greece / Kazakhstan 

I select "Greece" from path 'C'

*************************************************************************************************************************************
#### Final Path B: Bosnia and Herzegovina / Ukraine vs Israel / Iceland                  

I select "Iceland" from path 'B'

*************************************************************************************************************************************
#### Final Path A: Wales / Finland vs Poland / Estonia 

I select "Poland" from path 'A'

### CREATE TABLE SCHEMA

In [0]:
%sql
CREATE OR REPLACE TABLE team(name VARCHAR(15), grp VARCHAR(15));


### INSERT ALL TEAMS AND THEIR RESPECTED GROUPS IN IT

In [0]:
%sql
INSERT INTO team VALUES
    ('Germany','A'),
    ('Scotland','A'),
    ('Hungary','A'),
    ('Switzerland','A'),
    ('Spain','B'),
    ('Italy','B'),
    ('Croatia','B'),
    ('Albania','B'),
    ('England','C'),
    ('Slovenia','C'),
    ('Serbia','C'),
    ('Denmark','C'),
    ('Netherlands','D'),
    ('Austria','D'),
    ('France','D'),
    ('Poland','D'),
    ('Belgium','E'),
    ('Slovakia','E'),
    ('Romania','E'),
    ('Iceland','E'),
    ('Turkiye','F'),
    ('Portugal','F'),
    ('Czechia','F'),
    ('Greece','F');
    

num_affected_rows,num_inserted_rows
24,24


### HERE IS OUR INPUT TABLE OF 24 TEAMS AS PER EURO CUP 2024 FORMAT

In [0]:
%sql
SELECT * FROM team;


name,grp
Germany,A
Scotland,A
Hungary,A
Switzerland,A
Spain,B
Italy,B
Croatia,B
Albania,B
England,C
Slovenia,C


### STAGE 1 - GROUP STAGE MATCHES (36 MATCHES OF 24 TEAMS. EACH TEAM PLAYED 3 MATCH IN GROUP)

In [0]:
%sql
SELECT 
    CONCAT(a.name, " VS ", b.name) AS match_fixtures, 
    a.name AS team1, a.grp AS grp1, 
    b.name AS team2, b.grp AS grp2, 
    FLOOR(RAND()*5) AS team1_score, 
    FLOOR(RAND()*5) AS team2_score, 
    CASE 
        WHEN team1_score > team2_score THEN 3 
        WHEN team1_score = team2_score THEN 1 
        ELSE 0 
    END AS team1pts, 
    CASE 
        WHEN team1_score < team2_score THEN 3 
        WHEN team1_score = team2_score THEN 1 
        ELSE 0 
    END AS team2pts 
FROM 
    team a 
CROSS JOIN 
    team b 
ON 
    a.grp = b.grp AND a.name > b.name;
    

match_fixtures,team1,grp1,team2,grp2,team1_score,team2_score,team1pts,team2pts
Scotland VS Hungary,Scotland,A,Hungary,A,3,0,3,0
Scotland VS Germany,Scotland,A,Germany,A,4,3,3,0
Hungary VS Germany,Hungary,A,Germany,A,0,2,0,3
Switzerland VS Hungary,Switzerland,A,Hungary,A,3,4,0,3
Switzerland VS Scotland,Switzerland,A,Scotland,A,0,0,1,1
Switzerland VS Germany,Switzerland,A,Germany,A,1,2,0,3
Spain VS Albania,Spain,B,Albania,B,0,2,0,3
Spain VS Croatia,Spain,B,Croatia,B,0,4,0,3
Spain VS Italy,Spain,B,Italy,B,2,0,3,0
Italy VS Albania,Italy,B,Albania,B,2,3,0,3


### CREATING POINTS TABLE FROM ABOVE MATCH RESULTS AND GIVING RANKS TO TEAMS ACCORDING TO THEIR GROUPS

In [0]:
%sql
WITH cte2 AS (
    WITH cte AS (
        SELECT 
            CONCAT(a.name, " VS ", b.name) AS match_fixtures, 
            a.name AS team1, a.grp AS grp1, 
            b.name AS team2, b.grp AS grp2, 
            FLOOR(RAND()*5) AS team1_score, 
            FLOOR(RAND()*5) AS team2_score, 
            CASE 
                WHEN team1_score > team2_score THEN 3 
                WHEN team1_score = team2_score THEN 1 
                ELSE 0 
            END AS team1pts,
            CASE 
                WHEN team1_score < team2_score THEN 3 
                WHEN team1_score = team2_score THEN 1 
                ELSE 0 
            END AS team2pts 
        FROM 
            team a 
        CROSS JOIN 
            team b 
        ON 
            a.grp = b.grp AND a.name > b.name
    )
    SELECT 
        team1, grp1, team1_score, team2_score, team1pts 
    FROM 
        cte 
    UNION ALL 
    SELECT 
        team2, grp2, team1_score, team2_score, team2pts AS pts 
    FROM 
        cte
)
SELECT 
    team1 AS name, grp1 AS grp, 3 AS mp, 
    FLOOR(SUM(team1pts)/3) AS win, 
    CASE 
        WHEN SUM(team1pts) % 3 = 0 THEN 0 
        ELSE 1 
    END AS draw, 
    CASE 
        WHEN win + draw = 3 THEN 0 
        WHEN win + draw = 2 THEN 1 
        WHEN win + draw = 1 THEN 2 
        ELSE 3 
    END AS loss, 
    SUM(team1_score) AS gf, 
    SUM(team2_score) AS ga,
    SUM(team1_score) - SUM(team2_score) AS gd, 
    SUM(team1pts) AS pts,
    ROW_NUMBER() OVER(PARTITION BY grp1 ORDER BY SUM(team1pts) DESC,SUM(team1_score) - SUM(team2_score) DESC) rnk
FROM 
    cte2 
GROUP BY 
    team1, grp1 
ORDER BY 
    grp, pts DESC;
    

name,grp,mp,win,draw,loss,gf,ga,gd,pts,rnk
Germany,A,3,2,0,1,4,8,-4,6,1
Scotland,A,3,1,1,1,10,8,2,5,2
Switzerland,A,3,1,1,1,8,11,-3,4,3
Hungary,A,3,0,1,2,6,7,-1,1,4
Spain,B,3,2,1,0,11,7,4,7,1
Croatia,B,3,2,0,1,7,7,0,6,2
Albania,B,3,0,1,2,8,6,2,2,3
Italy,B,3,0,1,2,4,4,0,1,4
Slovenia,C,3,2,1,0,6,3,3,7,1
England,C,3,1,1,1,1,3,-2,4,2


### STAGE 2 - TOP 16 TEAMS FOR ROUND OF 16

In [0]:
%sql
WITH cte3 AS (
    WITH cte2 AS (
        WITH cte AS (
            SELECT 
                CONCAT(a.name, " VS ", b.name) AS match_fixtures, 
                a.name AS team1, a.grp AS grp1, 
                b.name AS team2, b.grp AS grp2, 
                FLOOR(RAND()*5) AS team1_score, 
                FLOOR(RAND()*5) AS team2_score, 
                CASE 
                    WHEN team1_score > team2_score THEN 3 
                    WHEN team1_score = team2_score THEN 1 
                    ELSE 0 
                END AS team1pts,
                CASE 
                    WHEN team1_score < team2_score THEN 3 
                    WHEN team1_score = team2_score THEN 1 
                    ELSE 0 
                END AS team2pts 
            FROM 
                team a 
            CROSS JOIN 
                team b 
            ON 
                a.grp = b.grp AND a.name > b.name
        )
        SELECT 
            team1, grp1, team1_score, team2_score, team1pts 
        FROM 
            cte 
        UNION ALL 
        SELECT 
            team2, grp2, team1_score, team2_score, team2pts AS pts 
        FROM 
            cte
    )
    SELECT 
        team1 AS name, grp1 AS grp, 3 AS mp, 
        FLOOR(SUM(team1pts)/3) AS win, 
        CASE 
            WHEN SUM(team1pts) % 3 = 0 THEN 0 
            ELSE 1 
        END AS draw, 
        CASE 
            WHEN win + draw = 3 THEN 0 
            WHEN win + draw = 2 THEN 1 
            WHEN win + draw = 1 THEN 2 
            ELSE 3 
        END AS loss, 
        SUM(team1_score) AS gf, 
        SUM(team2_score) AS ga,
        SUM(team1_score) - SUM(team2_score) AS gd, 
        SUM(team1pts) AS pts,
        ROW_NUMBER() OVER(PARTITION BY grp1 ORDER BY SUM(team1pts) DESC,SUM(team1_score) - SUM(team2_score) DESC) rnk
    FROM 
        cte2 
    GROUP BY 
        team1, grp1 
    ORDER BY 
        grp, pts DESC
)
SELECT * FROM cte3 WHERE rnk < 3
UNION
SELECT * FROM cte3 WHERE rnk = 3 ORDER BY rnk, pts DESC, gd DESC LIMIT 16;


name,grp,mp,win,draw,loss,gf,ga,gd,pts,rnk
England,C,3,2,1,0,10,7,3,7,1
Hungary,A,3,2,1,0,7,5,2,7,1
Czechia,F,3,2,1,0,7,9,-2,7,1
Netherlands,D,3,2,0,1,11,3,8,6,1
Italy,B,3,2,0,1,12,5,7,6,1
Romania,E,3,2,0,1,3,7,-4,6,1
France,D,3,2,0,1,9,4,5,6,2
Turkiye,F,3,2,0,1,11,7,4,6,2
Germany,A,3,2,0,1,7,7,0,6,2
Albania,B,3,2,0,1,7,7,0,6,2


### TAKING ONLY TEAM NAME,GROUP,RANK FOR EASY UNDERSTANDING OF FURTHER PROCESS AND PLACE THEM ACCORDING TO ROUND OF 16 MATCH FIXTURES

In [0]:
%sql
WITH cte5 AS (
    WITH cte4 AS (
        WITH cte3 AS (
            WITH cte2 AS (
                WITH cte AS (
                    SELECT 
                        CONCAT(a.name, " VS ", b.name) AS match_fixtures, 
                        a.name AS team1, a.grp AS grp1, 
                        b.name AS team2, b.grp AS grp2, 
                        FLOOR(RAND()*5) AS team1_score, 
                        FLOOR(RAND()*5) AS team2_score, 
                        CASE 
                            WHEN team1_score > team2_score THEN 3 
                            WHEN team1_score = team2_score THEN 1 
                            ELSE 0 
                        END AS team1pts,
                        CASE 
                            WHEN team1_score < team2_score THEN 3 
                            WHEN team1_score = team2_score THEN 1 
                            ELSE 0 
                        END AS team2pts 
                    FROM 
                        team a 
                    CROSS JOIN 
                        team b 
                    ON 
                        a.grp = b.grp AND a.name > b.name
                )
                SELECT 
                    team1, grp1, team1_score, team2_score, team1pts 
                FROM 
                    cte 
                UNION ALL 
                SELECT 
                    team2, grp2, team1_score, team2_score, team2pts AS pts 
                FROM 
                    cte
            )
            SELECT 
                team1 AS name, grp1 AS grp, 3 AS mp, 
                FLOOR(SUM(team1pts)/3) AS win, 
                CASE 
                    WHEN SUM(team1pts) % 3 = 0 THEN 0 
                    ELSE 1 
                END AS draw, 
                CASE 
                    WHEN win + draw = 3 THEN 0 
                    WHEN win + draw = 2 THEN 1 
                    WHEN win + draw = 1 THEN 2 
                    ELSE 3 
                END AS loss, 
                SUM(team1_score) AS gf, 
                SUM(team2_score) AS ga,
                SUM(team1_score) - SUM(team2_score) AS gd, 
                SUM(team1pts) AS pts,
                ROW_NUMBER() OVER(PARTITION BY grp1 ORDER BY SUM(team1pts) DESC,SUM(team1_score) - SUM(team2_score) DESC) rnk
            FROM 
                cte2 
            GROUP BY 
                team1, grp1 
            ORDER BY 
                grp, pts DESC
        )
        SELECT * FROM cte3 WHERE rnk < 3
        UNION
        SELECT * FROM cte3 WHERE rnk = 3 ORDER BY rnk, pts DESC, gd DESC LIMIT 16
    )
    SELECT name, grp, rnk FROM cte4 ORDER BY grp
)

SELECT name, grp, rnk FROM cte5 WHERE grp = 'B' AND rnk = 1
UNION

SELECT name, grp, rnk
FROM cte5
WHERE CASE 
      WHEN (SELECT concat_ws('', collect_list(grp))
            FROM cte5
            WHERE rnk = 3) IN ('ABCD', 'ABCE','ABCF') THEN (grp = 'A' AND rnk = 3)
      WHEN (SELECT concat_ws('', collect_list(grp))
            FROM cte5
            WHERE rnk = 3) IN ('ABDE', 'ABDF') THEN (grp = 'D' AND rnk = 3)
      WHEN (SELECT concat_ws('', collect_list(grp))
            FROM cte5
            WHERE rnk = 3) IN ('ABEF', 'ACDE','ACEF','ADEF','BCDE') THEN (grp = 'E' AND rnk = 3)
      WHEN (SELECT concat_ws('', collect_list(grp))
            FROM cte5
            WHERE rnk = 3) IN ('ACDF', 'BCDF','BCEF','BDEF','CDEF') THEN (grp = 'F' AND rnk = 3)
      END
UNION
SELECT name, grp, rnk FROM cte5 WHERE grp = 'A' AND rnk = 1
UNION
SELECT name, grp, rnk FROM cte5 WHERE grp = 'C' AND rnk = 2
UNION
SELECT name, grp, rnk FROM cte5 WHERE grp = 'F' AND rnk = 1
UNION

SELECT name, grp, rnk
FROM cte5
WHERE CASE 
      WHEN (SELECT concat_ws('', collect_list(grp))
            FROM cte5
            WHERE rnk = 3) IN ('ABEF','ACDE','ACDF','ACEF','ADEF') THEN (grp = 'A' AND rnk = 3)
      WHEN (SELECT concat_ws('', collect_list(grp))
            FROM cte5
            WHERE rnk = 3) IN ('ABDE','ABDF','BCDF','BCEF','BDEF') THEN (grp = 'B' AND rnk = 3)
      WHEN (SELECT concat_ws('', collect_list(grp))
            FROM cte5
            WHERE rnk = 3) IN ('ABCD','ABCE','ABCF','BCDE','CDEF') THEN (grp = 'C' AND rnk = 3)
      END
      
UNION
SELECT name, grp, rnk FROM cte5 WHERE grp = 'D' AND rnk = 2
UNION
SELECT name, grp, rnk FROM cte5 WHERE grp = 'E' AND rnk = 2
UNION
SELECT name, grp, rnk FROM cte5 WHERE grp = 'E' AND rnk = 1
UNION 

SELECT name, grp, rnk
FROM cte5
WHERE CASE 
      WHEN (SELECT concat_ws('', collect_list(grp))
            FROM cte5
            WHERE rnk = 3) IN ('ABDE', 'ABDF') THEN (grp = 'A' AND rnk = 3)
      WHEN (SELECT concat_ws('', collect_list(grp))
            FROM cte5
            WHERE rnk = 3) IN ('ABCD', 'ABCE','ABCF','ABEF','BCDE') THEN (grp = 'B' AND rnk = 3)
      WHEN (SELECT concat_ws('', collect_list(grp))
            FROM cte5
            WHERE rnk = 3) IN ('ACDE', 'ACDF','ACEF','BCDF','BCEF') THEN (grp = 'C' AND rnk = 3)
      WHEN (SELECT concat_ws('', collect_list(grp))
            FROM cte5
            WHERE rnk = 3) IN ('ADEF', 'BDEF','CDEF') THEN (grp = 'D' AND rnk = 3)
      END

UNION 
SELECT name, grp, rnk FROM cte5 WHERE grp = 'D' AND rnk = 1
UNION 
SELECT name, grp, rnk FROM cte5 WHERE grp = 'F' AND rnk = 2
UNION 
SELECT name, grp, rnk FROM cte5 WHERE grp = 'C' AND rnk = 1
UNION 

SELECT name, grp, rnk
FROM cte5
WHERE

 CASE 
      WHEN (SELECT concat_ws('', collect_list(grp))
            FROM cte5
            WHERE rnk = 3) IN ('ABCD','ACDE','ACDF','BCDE','BCDF') THEN (grp = 'D' AND rnk = 3)
      WHEN (SELECT concat_ws('', collect_list(grp))
            FROM cte5
            WHERE rnk = 3) IN ('ABCE','ABDE','BCEF','BDEF','CDEF') THEN (grp = 'E' AND rnk = 3)
      WHEN (SELECT concat_ws('', collect_list(grp))
            FROM cte5
            WHERE rnk = 3) IN ('ABCF','ABDF','ABEF','ACEF','ADEF') THEN (grp = 'F' AND rnk = 3)
      END

UNION 
SELECT name, grp, rnk FROM cte5 WHERE grp = 'A' AND rnk = 2
UNION 
SELECT name, grp, rnk FROM cte5 WHERE grp = 'B' AND rnk = 2;


name,grp,rnk
Croatia,B,1
Belgium,E,3
Scotland,A,1
Slovenia,C,2
Greece,F,1
Serbia,C,3
Netherlands,D,2
Slovakia,E,2
Iceland,E,1
Albania,B,3


### ROUND OF 16 MATCHES AND THEIR RESULTS (8 MATCHES OF 16 TEAMS)

In [0]:
%sql
WITH cte7 AS (
    WITH cte6 AS (
        WITH cte5 AS (
            WITH cte4 AS (
                WITH cte3 AS (
                    WITH cte2 AS (
                        WITH cte AS (
                            SELECT 
                                CONCAT(a.name, " VS ", b.name) AS match_fixtures, 
                                a.name AS team1, a.grp AS grp1, 
                                b.name AS team2, b.grp AS grp2, 
                                FLOOR(RAND()*5) AS team1_score, 
                                FLOOR(RAND()*5) AS team2_score, 
                                CASE 
                                    WHEN team1_score > team2_score THEN 3 
                                    WHEN team1_score = team2_score THEN 1 
                                    ELSE 0 
                                END AS team1pts,
                                CASE 
                                    WHEN team1_score < team2_score THEN 3 
                                    WHEN team1_score = team2_score THEN 1 
                                    ELSE 0 
                                END AS team2pts 
                            FROM 
                                team a 
                            CROSS JOIN 
                                team b 
                            ON 
                                a.grp = b.grp AND a.name > b.name
                        )
                        SELECT 
                            team1, grp1, team1_score, team2_score, team1pts 
                        FROM 
                            cte 
                        UNION ALL 
                        SELECT 
                            team2, grp2, team1_score, team2_score, team2pts AS pts 
                        FROM 
                            cte
                    )
                    SELECT 
                        team1 AS name, grp1 AS grp, 3 AS mp, 
                        FLOOR(SUM(team1pts)/3) AS win, 
                        CASE 
                            WHEN SUM(team1pts) % 3 = 0 THEN 0 
                            ELSE 1 
                        END AS draw, 
                        CASE 
                            WHEN win + draw = 3 THEN 0 
                            WHEN win + draw = 2 THEN 1 
                            WHEN win + draw = 1 THEN 2 
                            ELSE 3 
                        END AS loss, 
                        SUM(team1_score) AS gf, 
                        SUM(team2_score) AS ga,
                        SUM(team1_score) - SUM(team2_score) AS gd, 
                        SUM(team1pts) AS pts,
                        ROW_NUMBER() OVER(PARTITION BY grp1 ORDER BY SUM(team1pts) DESC,SUM(team1_score) - SUM(team2_score) DESC) rnk
                    FROM 
                        cte2 
                    GROUP BY 
                        team1, grp1 
                    ORDER BY 
                        grp, pts DESC
                )
                SELECT * FROM cte3 WHERE rnk < 3
                UNION
                SELECT * FROM cte3 WHERE rnk = 3 ORDER BY rnk, pts DESC, gd DESC LIMIT 16
            )
            SELECT name, grp, rnk FROM cte4 ORDER BY grp
        )

        SELECT 
            name, grp, rnk 
        FROM 
            cte5 
        WHERE 
            grp = 'B' AND rnk = 1
        UNION

        SELECT 
            name, grp, rnk
        FROM 
            cte5
        WHERE 
            CASE 
                WHEN (SELECT CONCAT_WS('', COLLECT_LIST(grp))
                    FROM cte5
                    WHERE rnk = 3) IN ('ABCD', 'ABCE','ABCF') THEN (grp = 'A' AND rnk = 3)
                WHEN (SELECT CONCAT_WS('', COLLECT_LIST(grp))
                    FROM cte5
                    WHERE rnk = 3) IN ('ABDE', 'ABDF') THEN (grp = 'D' AND rnk = 3)
                WHEN (SELECT CONCAT_WS('', COLLECT_LIST(grp))
                    FROM cte5
                    WHERE rnk = 3) IN ('ABEF', 'ACDE','ACEF','ADEF','BCDE') THEN (grp = 'E' AND rnk = 3)
                WHEN (SELECT CONCAT_WS('', COLLECT_LIST(grp))
                    FROM cte5
                    WHERE rnk = 3) IN ('ACDF', 'BCDF','BCEF','BDEF','CDEF') THEN (grp = 'F' AND rnk = 3)
            END

        UNION

        SELECT 
            name, grp, rnk 
        FROM 
            cte5 
        WHERE 
            grp = 'A' AND rnk = 1
        UNION
        SELECT 
            name, grp, rnk 
        FROM 
            cte5 
        WHERE 
            grp = 'C' AND rnk = 2
        UNION
        SELECT 
            name, grp, rnk 
        FROM 
            cte5 
        WHERE 
            grp = 'F' AND rnk = 1
        UNION

        SELECT 
            name, grp, rnk
        FROM 
            cte5
        WHERE 
            CASE 
                WHEN (SELECT CONCAT_WS('', COLLECT_LIST(grp))
                    FROM cte5
                    WHERE rnk = 3) IN ('ABEF','ACDE','ACDF','ACEF','ADEF') THEN (grp = 'A' AND rnk = 3)
                WHEN (SELECT CONCAT_WS('', COLLECT_LIST(grp))
                    FROM cte5
                    WHERE rnk = 3) IN ('ABDE','ABDF','BCDF','BCEF','BDEF') THEN (grp = 'B' AND rnk = 3)
                WHEN (SELECT CONCAT_WS('', COLLECT_LIST(grp))
                    FROM cte5
                    WHERE rnk = 3) IN ('ABCD','ABCE','ABCF','BCDE','CDEF') THEN (grp = 'C' AND rnk = 3)
            END

        UNION

        SELECT 
            name, grp, rnk 
        FROM 
            cte5 
        WHERE 
            grp = 'D' AND rnk = 2
        UNION
        SELECT 
            name, grp, rnk 
        FROM 
            cte5 
        WHERE 
            grp = 'E' AND rnk = 2
        UNION
        SELECT 
            name, grp, rnk 
        FROM 
            cte5 
        WHERE 
            grp = 'E' AND rnk = 1
        UNION 

        SELECT 
            name, grp, rnk
        FROM 
            cte5
        WHERE 
            CASE 
                WHEN (SELECT CONCAT_WS('', COLLECT_LIST(grp))
                    FROM cte5
                    WHERE rnk = 3) IN ('ABDE', 'ABDF') THEN (grp = 'A' AND rnk = 3)
                WHEN (SELECT CONCAT_WS('', COLLECT_LIST(grp))
                    FROM cte5
                    WHERE rnk = 3) IN ('ABCD', 'ABCE','ABCF','ABEF','BCDE') THEN (grp = 'B' AND rnk = 3)
                WHEN (SELECT CONCAT_WS('', COLLECT_LIST(grp))
                    FROM cte5
                    WHERE rnk = 3) IN ('ACDE', 'ACDF','ACEF','BCDF','BCEF') THEN (grp = 'C' AND rnk = 3)
                WHEN (SELECT CONCAT_WS('', COLLECT_LIST(grp))
                    FROM cte5
                    WHERE

 rnk = 3) IN ('ADEF', 'BDEF','CDEF') THEN (grp = 'D' AND rnk = 3)
            END

        UNION 
        SELECT 
            name, grp, rnk 
        FROM 
            cte5 
        WHERE 
            grp = 'D' AND rnk = 1
        UNION 
        SELECT 
            name, grp, rnk 
        FROM 
            cte5 
        WHERE 
            grp = 'F' AND rnk = 2
        UNION 
        SELECT 
            name, grp, rnk 
        FROM 
            cte5 
        WHERE 
            grp = 'C' AND rnk = 1
        UNION 

        SELECT 
            name, grp, rnk
        FROM 
            cte5
        WHERE 
            CASE 
                WHEN (SELECT CONCAT_WS('', COLLECT_LIST(grp))
                    FROM cte5
                    WHERE rnk = 3) IN ('ABCD','ACDE','ACDF','BCDE','BCDF') THEN (grp = 'D' AND rnk = 3)
                WHEN (SELECT CONCAT_WS('', COLLECT_LIST(grp))
                    FROM cte5
                    WHERE rnk = 3) IN ('ABCE','ABDE','BCEF','BDEF','CDEF') THEN (grp = 'E' AND rnk = 3)
                WHEN (SELECT CONCAT_WS('', COLLECT_LIST(grp))
                    FROM cte5
                    WHERE rnk = 3) IN ('ABCF','ABDF','ABEF','ACEF','ADEF') THEN (grp = 'F' AND rnk = 3)
            END

        UNION 
        SELECT 
            name, grp, rnk 
        FROM 
            cte5 
        WHERE 
            grp = 'A' AND rnk = 2
        UNION 
        SELECT 
            name, grp, rnk 
        FROM 
            cte5 
        WHERE 
            grp = 'B' AND rnk = 2
    )
    SELECT 
        *, NTILE(8) OVER (ORDER BY (SELECT 0)) AS matches 
    FROM 
        cte6
)
SELECT 
    CONCAT(a.name," ",a.grp,a.rnk,"    VS    ",b.name," ",b.grp,b.rnk) AS round_16_fixtures,
    a.matches, 
    CASE 
        WHEN ROUND(RAND()*2) = 0 THEN "WIN" 
        ELSE "LOSE" 
    END AS team_1, 
    CASE 
        WHEN team_1 = "WIN" THEN "LOSE" 
        ELSE "WIN" 
    END AS team_2,
    CASE 
        WHEN team_1 = "WIN" THEN CONCAT(a.name," ",a.grp,a.rnk) 
        ELSE CONCAT(b.name," ",b.grp,b.rnk) 
    END AS quarter_finalist 
FROM 
    cte7 a 
INNER JOIN 
    cte7 b ON a.matches = b.matches AND a.name > b.name;


round_16_fixtures,matches,team_1,team_2,quarter_finalist
Czechia F3 VS Albania B1,1,LOSE,WIN,Albania B1
Slovenia C2 VS Hungary A1,2,LOSE,WIN,Hungary A1
Greece F1 VS Denmark C3,3,WIN,LOSE,Greece F1
Iceland E2 VS Austria D2,4,LOSE,WIN,Austria D2
France D3 VS Belgium E1,5,WIN,LOSE,France D3
Portugal F2 VS Poland D1,6,LOSE,WIN,Poland D1
Slovakia E3 VS Serbia C1,7,WIN,LOSE,Slovakia E3
Scotland A2 VS Croatia B2,8,WIN,LOSE,Scotland A2


### STAGE 3 - QUARTER-FINAL MATCHES AND THEIR RESULTS (4 MATCHES OF 8 TEAMS)

In [0]:
%sql
WITH cte9 AS (
    WITH cte8 AS (
        WITH cte7 AS (
            WITH cte6 AS (
                WITH cte5 AS (
                    WITH cte4 AS (
                        WITH cte3 AS (
                            WITH cte2 AS (
                                WITH cte AS (
                                    SELECT 
                                        CONCAT(a.name, " VS ", b.name) AS match_fixtures, 
                                        a.name AS team1, a.grp AS grp1, 
                                        b.name AS team2, b.grp AS grp2, 
                                        FLOOR(RAND()*5) AS team1_score, 
                                        FLOOR(RAND()*5) AS team2_score, 
                                        CASE 
                                            WHEN team1_score > team2_score THEN 3 
                                            WHEN team1_score = team2_score THEN 1 
                                            ELSE 0 
                                        END AS team1pts,
                                        CASE 
                                            WHEN team1_score < team2_score THEN 3 
                                            WHEN team1_score = team2_score THEN 1 
                                            ELSE 0 
                                        END AS team2pts 
                                    FROM 
                                        team a 
                                    CROSS JOIN 
                                        team b 
                                    ON 
                                        a.grp = b.grp AND a.name > b.name
                                )
                                SELECT 
                                    team1, grp1, team1_score, team2_score, team1pts 
                                FROM 
                                    cte 
                                UNION ALL 
                                SELECT 
                                    team2, grp2, team1_score, team2_score, team2pts AS pts 
                                FROM 
                                    cte
                            )
                            SELECT 
                                team1 AS name, grp1 AS grp, 3 AS mp, 
                                FLOOR(SUM(team1pts)/3) AS win, 
                                CASE 
                                    WHEN SUM(team1pts) % 3 = 0 THEN 0 
                                    ELSE 1 
                                END AS draw, 
                                CASE 
                                    WHEN win + draw = 3 THEN 0 
                                    WHEN win + draw = 2 THEN 1 
                                    WHEN win + draw = 1 THEN 2 
                                    ELSE 3 
                                END AS loss, 
                                SUM(team1_score) AS gf, 
                                SUM(team2_score) AS ga,
                                SUM(team1_score) - SUM(team2_score) AS gd, 
                                SUM(team1pts) AS pts,
                                ROW_NUMBER() OVER(PARTITION BY grp1 ORDER BY SUM(team1pts) DESC,SUM(team1_score) - SUM(team2_score) DESC) rnk
                            FROM 
                                cte2 
                            GROUP BY 
                                team1, grp1 
                            ORDER BY 
                                grp, pts DESC
                        )
                        SELECT * FROM cte3 WHERE rnk < 3
                        UNION
                        SELECT * FROM cte3 WHERE rnk = 3 ORDER BY rnk, pts DESC, gd DESC LIMIT 16
                    )
                    SELECT name, grp, rnk FROM cte4 ORDER BY grp
                )

                SELECT 
                    name, grp, rnk 
                FROM 
                    cte5 
                WHERE 
                    grp = 'B' AND rnk = 1
                UNION

                SELECT 
                    name, grp, rnk
                FROM 
                    cte5
                WHERE 
                    CASE 
                        WHEN (SELECT CONCAT_WS('', COLLECT_LIST(grp))
                            FROM cte5
                            WHERE rnk = 3) IN ('ABCD', 'ABCE','ABCF') THEN (grp = 'A' AND rnk = 3)
                        WHEN (SELECT CONCAT_WS('', COLLECT_LIST(grp))
                            FROM cte5
                            WHERE rnk = 3) IN ('ABDE', 'ABDF') THEN (grp = 'D' AND rnk = 3)
                        WHEN (SELECT CONCAT_WS('', COLLECT_LIST(grp))
                            FROM cte5
                            WHERE rnk = 3) IN ('ABEF', 'ACDE','ACEF','ADEF','BCDE') THEN (grp = 'E' AND rnk = 3)
                        WHEN (SELECT CONCAT_WS('', COLLECT_LIST(grp))
                            FROM cte5
                            WHERE rnk = 3) IN ('ACDF', 'BCDF','BCEF','BDEF','CDEF') THEN (grp = 'F' AND rnk = 3)
                    END

                UNION

                SELECT 
                    name, grp, rnk 
                FROM 
                    cte5 
                WHERE 
                    grp = 'A' AND rnk = 1
                UNION
                SELECT 
                    name, grp, rnk 
                FROM 
                    cte5 
                WHERE 
                    grp = 'C' AND rnk = 2
                UNION
                SELECT 
                    name, grp, rnk 
                FROM 
                    cte5 
                WHERE 
                    grp = 'F' AND rnk = 1
                UNION

                SELECT 
                    name, grp, rnk
                FROM 
                    cte5
                WHERE 
                    CASE 
                        WHEN (SELECT CONCAT_WS('', COLLECT_LIST(grp))
                            FROM cte5
                            WHERE rnk = 3) IN ('ABEF','ACDE','ACDF','ACEF','ADEF') THEN (grp = 'A' AND rnk = 3)
                        WHEN (SELECT CONCAT_WS('', COLLECT_LIST(grp))
                            FROM cte5
                            WHERE rnk = 3) IN ('ABDE','ABDF','BCDF','BCEF','BDEF') THEN (grp = 'B' AND rnk = 3)
                        WHEN (SELECT CONCAT_WS('', COLLECT_LIST(grp))
                            FROM cte5
                            WHERE rnk = 3) IN ('ABCD','ABCE','ABCF','BCDE','CDEF') THEN (grp = 'C' AND rnk = 3)
                    END

                UNION

                SELECT 
                    name, grp, rnk 
                FROM 
                    cte5 
                WHERE 
                    grp = 'D' AND rnk = 2
                UNION
                SELECT 
                    name, grp, rnk 
                FROM 
                    cte5 
                WHERE 
                    grp = 'E' AND rnk = 2
                UNION
                SELECT 
                    name, grp, rnk 
                FROM 
                    cte5 
                WHERE 
                    grp = 'E' AND rnk = 1
                UNION 

                SELECT 
                    name, grp, rnk
                FROM 
                    cte5
                WHERE 
                    CASE 
                        WHEN (SELECT CONCAT_WS('', COLLECT_LIST(grp))
                            FROM cte5
                            WHERE rnk = 3) IN ('ABDE', 'ABDF') THEN (grp = 'A' AND rnk = 3)
                        WHEN (SELECT CONCAT_WS('', COLLECT_LIST(grp))
                            FROM cte5
                            WHERE rnk = 3) IN ('ABCD', 'ABCE','ABCF','ABEF','BCDE') THEN (grp = 'B' AND rnk = 3)
                        WHEN (SELECT CONCAT_WS('', COLLECT_LIST(grp))
                            FROM cte5
                            WHERE rnk = 3) IN ('ACDE', 'ACDF','ACEF','BCDF','BCEF') THEN (grp = 'C' AND rnk = 3)
                        WHEN (SELECT CONCAT_WS('',

 COLLECT_LIST(grp))
                            FROM cte5
                            WHERE rnk = 3) IN ('ADEF', 'BDEF','CDEF') THEN (grp = 'D' AND rnk = 3)
                    END

                UNION 
                SELECT 
                    name, grp, rnk 
                FROM 
                    cte5 
                WHERE 
                    grp = 'D' AND rnk = 1
                UNION 
                SELECT 
                    name, grp, rnk 
                FROM 
                    cte5 
                WHERE 
                    grp = 'F' AND rnk = 2
                UNION 
                SELECT 
                    name, grp, rnk 
                FROM 
                    cte5 
                WHERE 
                    grp = 'C' AND rnk = 1
                UNION 

                SELECT 
                    name, grp, rnk
                FROM 
                    cte5
                WHERE 
                    CASE 
                        WHEN (SELECT CONCAT_WS('', COLLECT_LIST(grp))
                            FROM cte5
                            WHERE rnk = 3) IN ('ABCD','ACDE','ACDF','BCDE','BCDF') THEN (grp = 'D' AND rnk = 3)
                        WHEN (SELECT CONCAT_WS('', COLLECT_LIST(grp))
                            FROM cte5
                            WHERE rnk = 3) IN ('ABCE','ABDE','BCEF','BDEF','CDEF') THEN (grp = 'E' AND rnk = 3)
                        WHEN (SELECT CONCAT_WS('', COLLECT_LIST(grp))
                            FROM cte5
                            WHERE rnk = 3) IN ('ABCF','ABDF','ABEF','ACEF','ADEF') THEN (grp = 'F' AND rnk = 3)
                    END

                UNION 
                SELECT 
                    name, grp, rnk 
                FROM 
                    cte5 
                WHERE 
                    grp = 'A' AND rnk = 2
                UNION 
                SELECT 
                    name, grp, rnk 
                FROM 
                    cte5 
                WHERE 
                    grp = 'B' AND rnk = 2
            )
            SELECT 
                *, NTILE(8) OVER (ORDER BY (SELECT 0)) AS matches 
            FROM 
                cte6
        )
        SELECT 
            CONCAT(a.name," ",a.grp,a.rnk,"    VS    ",b.name," ",b.grp,b.rnk) AS round_16_fixtures,
            a.matches, 
            CASE 
                WHEN ROUND(RAND()*2) = 0 THEN "WIN" 
                ELSE "LOSE" 
            END AS team_1, 
            CASE 
                WHEN team_1 = "WIN" THEN "LOSE" 
                ELSE "WIN" 
            END AS team_2,
            CASE 
                WHEN team_1 = "WIN" THEN CONCAT(a.name," ",a.grp,a.rnk) 
                ELSE CONCAT(b.name," ",b.grp,b.rnk) 
            END AS quarter_finalist 
        FROM 
            cte7 a 
        INNER JOIN 
            cte7 b ON a.matches = b.matches AND a.name > b.name
    )
    SELECT 
        quarter_finalist, NTILE(4) OVER (ORDER BY (SELECT 0)) AS matches 
    FROM 
        cte8
)
SELECT 
    CONCAT(a.quarter_finalist,"   VS   ",b.quarter_finalist) AS quarter_final_fixtures,
    a.matches,
    CASE 
        WHEN ROUND(RAND()*2) = 0 THEN "WIN" 
        ELSE "LOSE" 
    END AS team_1, 
    CASE 
        WHEN team_1 = "WIN" THEN "LOSE" 
        ELSE "WIN" 
    END AS team_2,
    CASE 
        WHEN team_1 = "WIN" THEN a.quarter_finalist 
        ELSE b.quarter_finalist 
    END AS semi_finalist 
FROM 
    cte9 a 
INNER JOIN 
    cte9 b ON a.matches = b.matches AND a.quarter_finalist > b.quarter_finalist 
ORDER BY 
    a.matches;


quarter_final_fixtures,matches,team_1,team_2,semi_finalist
Denmark C2 VS Czechia F3,1,LOSE,WIN,Czechia F3
Portugal F1 VS France D2,2,LOSE,WIN,France D2
Slovenia C3 VS Poland D1,3,WIN,LOSE,Slovenia C3
Hungary A2 VS Austria D3,4,LOSE,WIN,Austria D3


### STAGE 4 - SEMI-FIANL MATCHES AND THEIR RESULTS (2 MATCHES OF 4 TEAMS)

In [0]:
%sql
WITH cte11 AS (
    WITH cte10 AS (
        WITH cte9 AS (
            WITH cte8 AS (
                WITH cte7 AS (
                    WITH cte6 AS (
                        WITH cte5 AS (
                            WITH cte4 AS (
                                WITH cte3 AS (
                                    WITH cte2 AS (
                                        WITH cte AS (
                                            SELECT 
                                                CONCAT(a.name, " VS ", b.name) AS match_fixtures, 
                                                a.name AS team1, a.grp AS grp1, 
                                                b.name AS team2, b.grp AS grp2, 
                                                FLOOR(RAND()*5) AS team1_score, 
                                                FLOOR(RAND()*5) AS team2_score, 
                                                CASE 
                                                    WHEN team1_score > team2_score THEN 3 
                                                    WHEN team1_score = team2_score THEN 1 
                                                    ELSE 0 
                                                END AS team1pts,
                                                CASE 
                                                    WHEN team1_score < team2_score THEN 3 
                                                    WHEN team1_score = team2_score THEN 1 
                                                    ELSE 0 
                                                END AS team2pts 
                                            FROM 
                                                team a 
                                            CROSS JOIN 
                                                team b 
                                            ON 
                                                a.grp = b.grp AND a.name > b.name
                                        )
                                        SELECT 
                                            team1, grp1, team1_score, team2_score, team1pts 
                                        FROM 
                                            cte 
                                        UNION ALL 
                                        SELECT 
                                            team2, grp2, team1_score, team2_score, team2pts AS pts 
                                        FROM 
                                            cte
                                    )
                                    SELECT 
                                        team1 AS name, grp1 AS grp, 3 AS mp, 
                                        FLOOR(SUM(team1pts)/3) AS win, 
                                        CASE 
                                            WHEN SUM(team1pts) % 3 = 0 THEN 0 
                                            ELSE 1 
                                        END AS draw, 
                                        CASE 
                                            WHEN win + draw = 3 THEN 0 
                                            WHEN win + draw = 2 THEN 1 
                                            WHEN win + draw = 1 THEN 2 
                                            ELSE 3 
                                        END AS loss, 
                                        SUM(team1_score) AS gf, 
                                        SUM(team2_score) AS ga,
                                        SUM(team1_score) - SUM(team2_score) AS gd, 
                                        SUM(team1pts) AS pts,
                                        ROW_NUMBER() OVER(PARTITION BY grp1 ORDER BY SUM(team1pts) DESC,SUM(team1_score) - SUM(team2_score) DESC) rnk
                                    FROM 
                                        cte2 
                                    GROUP BY 
                                        team1, grp1 
                                    ORDER BY 
                                        grp, pts DESC
                                )
                                SELECT * FROM cte3 WHERE rnk < 3
                                UNION
                                SELECT * FROM cte3 WHERE rnk = 3 ORDER BY rnk, pts DESC, gd DESC LIMIT 16
                            )
                            SELECT name, grp, rnk FROM cte4 ORDER BY grp
                        )

                        SELECT 
                            name, grp, rnk 
                        FROM 
                            cte5 
                        WHERE 
                            grp = 'B' AND rnk = 1
                        UNION

                        SELECT 
                            name, grp, rnk
                        FROM 
                            cte5
                        WHERE 
                            CASE 
                                WHEN (SELECT CONCAT_WS('', COLLECT_LIST(grp))
                                    FROM cte5
                                    WHERE rnk = 3) IN ('ABCD', 'ABCE','ABCF') THEN (grp = 'A' AND rnk = 3)
                                WHEN (SELECT CONCAT_WS('', COLLECT_LIST(grp))
                                    FROM cte5
                                    WHERE rnk = 3) IN ('ABDE', 'ABDF') THEN (grp = 'D' AND rnk = 3)
                                WHEN (SELECT CONCAT_WS('', COLLECT_LIST(grp))
                                    FROM cte5
                                    WHERE rnk = 3) IN ('ABEF', 'ACDE','ACEF','ADEF','BCDE') THEN (grp = 'E' AND rnk = 3)
                                WHEN (SELECT CONCAT_WS('', COLLECT_LIST(grp))
                                    FROM cte5
                                    WHERE rnk = 3) IN ('ACDF', 'BCDF','BCEF','BDEF','CDEF') THEN (grp = 'F' AND rnk = 3)
                            END

                        UNION

                        SELECT 
                            name, grp, rnk 
                        FROM 
                            cte5 
                        WHERE 
                            grp = 'A' AND rnk = 1
                        UNION
                        SELECT 
                            name, grp, rnk 
                        FROM 
                            cte5 
                        WHERE 
                            grp = 'C' AND rnk = 2
                        UNION
                        SELECT 
                            name, grp, rnk 
                        FROM 
                            cte5 
                        WHERE 
                            grp = 'F' AND rnk = 1
                        UNION

                        SELECT 
                            name, grp, rnk
                        FROM 
                            cte5
                        WHERE 
                            CASE 
                                WHEN (SELECT CONCAT_WS('', COLLECT_LIST(grp))
                                    FROM cte5
                                    WHERE rnk = 3) IN ('ABEF','ACDE','ACDF','ACEF','ADEF') THEN (grp = 'A' AND rnk = 3)
                                WHEN (SELECT CONCAT_WS('', COLLECT_LIST(grp))
                                    FROM cte5
                                    WHERE rnk = 3) IN ('ABDE','ABDF','BCDF','BCEF','BDEF') THEN (grp = 'B' AND rnk = 3)
                                WHEN (SELECT CONCAT_WS('', COLLECT_LIST(grp))
                                    FROM cte5
                                   

 WHERE rnk = 3) IN ('ABCD','ABCE','ABCF','BCDE','CDEF') THEN (grp = 'C' AND rnk = 3)
                            END

                        UNION
                        SELECT 
                            name, grp, rnk 
                        FROM 
                            cte5 
                        WHERE 
                            grp = 'D' AND rnk = 2
                        UNION
                        SELECT 
                            name, grp, rnk 
                        FROM 
                            cte5 
                        WHERE 
                            grp = 'E' AND rnk = 2
                        UNION
                        SELECT 
                            name, grp, rnk 
                        FROM 
                            cte5 
                        WHERE 
                            grp = 'E' AND rnk = 1
                        UNION 

                        SELECT 
                            name, grp, rnk
                        FROM 
                            cte5
                        WHERE 
                            CASE 
                                WHEN (SELECT CONCAT_WS('', COLLECT_LIST(grp))
                                    FROM cte5
                                    WHERE rnk = 3) IN ('ABDE', 'ABDF') THEN (grp = 'A' AND rnk = 3)
                                WHEN (SELECT CONCAT_WS('', COLLECT_LIST(grp))
                                    FROM cte5
                                    WHERE rnk = 3) IN ('ABCD', 'ABCE','ABCF','ABEF','BCDE') THEN (grp = 'B' AND rnk = 3)
                                WHEN (SELECT CONCAT_WS('', COLLECT_LIST(grp))
                                    FROM cte5
                                    WHERE rnk = 3) IN ('ACDE', 'ACDF','ACEF','BCDF','BCEF') THEN (grp = 'C' AND rnk = 3)
                                WHEN (SELECT CONCAT_WS('', COLLECT_LIST(grp))
                                    FROM cte5
                                    WHERE rnk = 3) IN ('ADEF', 'BDEF','CDEF') THEN (grp = 'D' AND rnk = 3)
                            END

                        UNION 
                        SELECT 
                            name, grp, rnk 
                        FROM 
                            cte5 
                        WHERE 
                            grp = 'D' AND rnk = 1
                        UNION 
                        SELECT 
                            name, grp, rnk 
                        FROM 
                            cte5 
                        WHERE 
                            grp = 'F' AND rnk = 2
                        UNION 
                        SELECT 
                            name, grp, rnk 
                        FROM 
                            cte5 
                        WHERE 
                            grp = 'C' AND rnk = 1
                        UNION 

                        SELECT 
                            name, grp, rnk
                        FROM 
                            cte5
                        WHERE 
                            CASE 
                                WHEN (SELECT CONCAT_WS('', COLLECT_LIST(grp))
                                    FROM cte5
                                    WHERE rnk = 3) IN ('ABCD','ACDE','ACDF','BCDE','BCDF') THEN (grp = 'D' AND rnk = 3)
                                WHEN (SELECT CONCAT_WS('', COLLECT_LIST(grp))
                                    FROM cte5
                                    WHERE rnk = 3) IN ('ABCE','ABDE','BCEF','BDEF','CDEF') THEN (grp = 'E' AND rnk = 3)
                                WHEN (SELECT CONCAT_WS('', COLLECT_LIST(grp))
                                    FROM cte5
                                    WHERE rnk = 3) IN ('ABCF','ABDF','ABEF','ACEF','ADEF') THEN (grp = 'F' AND rnk = 3)
                            END

                        UNION 
                        SELECT 
                            name, grp, rnk 
                        FROM 
                            cte5 
                        WHERE 
                            grp = 'A' AND rnk = 2
                        UNION 
                        SELECT 
                            name, grp, rnk 
                        FROM 
                            cte5 
                        WHERE 
                            grp = 'B' AND rnk = 2
                    )
                    SELECT 
                        *, NTILE(8) OVER (ORDER BY (SELECT 0)) AS matches 
                    FROM 
                        cte6
                )
                SELECT 
                    CONCAT(a.name," ",a.grp,a.rnk,"    VS    ",b.name," ",b.grp,b.rnk) AS round_16_fixtures,
                    a.matches, 
                    CASE 
                        WHEN ROUND(RAND()*2) = 0 THEN "WIN" 
                        ELSE "LOSE" 
                    END AS team_1, 
                    CASE 
                        WHEN team_1 = "WIN" THEN "LOSE" 
                        ELSE "WIN" 
                    END AS team_2,
                    CASE 
                        WHEN team_1 = "WIN" THEN CONCAT(a.name," ",a.grp,a.rnk) 
                        ELSE CONCAT(b.name," ",b.grp,b.rnk) 
                    END AS quarter_finalist 
                FROM 
                    cte7 a 
                INNER JOIN 
                    cte7 b ON a.matches = b.matches AND a.name > b.name
            )
            SELECT 
                quarter_finalist, NTILE(4) OVER (ORDER BY (SELECT 0)) AS matches 
            FROM 
                cte8
        )
        SELECT 
            CONCAT(a.quarter_finalist,"   VS   ",b.quarter_finalist) AS quarter_final_fixtures,
            a.matches,
            CASE 
                WHEN ROUND(RAND()*2) = 0 THEN "WIN" 
                ELSE "LOSE" 
            END AS team_1, 
            CASE 
                WHEN team_1 = "WIN" THEN "LOSE" 
                ELSE "WIN" 
            END AS team_2,
            CASE 
                WHEN team_1 = "WIN" THEN a.quarter_finalist 
                ELSE b.quarter_finalist 
            END AS semi_finalist 
        FROM 
            cte9 a 
        INNER JOIN 
            cte9 b ON a.matches = b.matches AND a.quarter_finalist > b.quarter_finalist 
        ORDER BY 
            a.matches
    )
    SELECT 
        semi_finalist, NTILE(2) OVER (ORDER BY (SELECT 0)) AS matches 
    FROM 
        cte10
)
SELECT 
    CONCAT(a.semi_finalist,"   VS   ",b.semi_finalist) AS semi_final_fixtures,
    a.matches,
    CASE 
        WHEN ROUND(RAND()*2) = 0 THEN "WIN" 
        ELSE "LOSE" 
    END AS team_1, 
    CASE 
        WHEN team_1 = "WIN" THEN "LOSE" 
        ELSE "WIN" 
    END AS team_2,
    CASE 
        WHEN team_1 = "WIN" THEN a.semi_finalist 
        ELSE b.semi_finalist 
    END AS finalist 
FROM 
    cte11 a 
INNER JOIN 
    cte11 b ON a.matches = b.matches AND a.semi_finalist > b.semi_finalist 
ORDER BY 
    a.matches;


semi_final_fixtures,matches,team_1,team_2,finalist
Spain B1 VS Denmark C3,1,LOSE,WIN,Denmark C3
Belgium E1 VS Albania B2,2,LOSE,WIN,Albania B2


### STAGE 5 - FINAL MATCH AND ITS RESULT

In [0]:
%sql
WITH cte13 AS (
    WITH cte12 AS (
        WITH cte11 AS (
            WITH cte10 AS (
                WITH cte9 AS (
                    WITH cte8 AS (
                        WITH cte7 AS (
                            WITH cte6 AS (
                                WITH cte5 AS (
                                    WITH cte4 AS (
                                        WITH cte3 AS (
                                            WITH cte2 AS (
                                                WITH cte AS (
                                                    SELECT 
                                                        CONCAT(a.name, " VS ", b.name) AS match_fixtures, 
                                                        a.name AS team1, a.grp AS grp1, 
                                                        b.name AS team2, b.grp AS grp2, 
                                                        FLOOR(RAND()*5) AS team1_score, 
                                                        FLOOR(RAND()*5) AS team2_score, 
                                                        CASE 
                                                            WHEN team1_score > team2_score THEN 3 
                                                            WHEN team1_score = team2_score THEN 1 
                                                            ELSE 0 
                                                        END AS team1pts,
                                                        CASE 
                                                            WHEN team1_score < team2_score THEN 3 
                                                            WHEN team1_score = team2_score THEN 1 
                                                            ELSE 0 
                                                        END AS team2pts 
                                                    FROM 
                                                        team a 
                                                    CROSS JOIN 
                                                        team b 
                                                    ON 
                                                        a.grp = b.grp AND a.name > b.name
                                                )
                                                SELECT 
                                                    team1, grp1, team1_score, team2_score, team1pts 
                                                FROM 
                                                    cte 
                                                UNION ALL 
                                                SELECT 
                                                    team2, grp2, team1_score, team2_score, team2pts AS pts 
                                                FROM 
                                                    cte
                                            )
                                            SELECT 
                                                team1 AS name, grp1 AS grp, 3 AS mp, 
                                                FLOOR(SUM(team1pts)/3) AS win, 
                                                CASE 
                                                    WHEN SUM(team1pts) % 3 = 0 THEN 0 
                                                    ELSE 1 
                                                END AS draw, 
                                                CASE 
                                                    WHEN win + draw = 3 THEN 0 
                                                    WHEN win + draw = 2 THEN 1 
                                                    WHEN win + draw = 1 THEN 2 
                                                    ELSE 3 
                                                END AS loss, 
                                                SUM(team1_score) AS gf, 
                                                SUM(team2_score) AS ga,
                                                SUM(team1_score) - SUM(team2_score) AS gd, 
                                                SUM(team1pts) AS pts,
                                                ROW_NUMBER() OVER(PARTITION BY grp1 ORDER BY SUM(team1pts) DESC,SUM(team1_score) - SUM(team2_score) DESC) rnk
                                            FROM 
                                                cte2 
                                            GROUP BY 
                                                team1, grp1 
                                            ORDER BY 
                                                grp, pts DESC
                                        )
                                        SELECT * FROM cte3 WHERE rnk < 3
                                        UNION
                                        SELECT * FROM cte3 WHERE rnk = 3 ORDER BY rnk, pts DESC, gd DESC LIMIT 16
                                    )
                                    SELECT name, grp, rnk FROM cte4 ORDER BY grp
                                )
                                SELECT name, grp, rnk FROM cte5 WHERE grp = 'B' AND rnk = 1
                                UNION
                                SELECT name, grp, rnk
                                FROM cte5
                                WHERE CASE 
                                      WHEN (SELECT concat_ws('', collect_list(grp))
                                            FROM cte5
                                            WHERE rnk = 3) IN ('ABCD', 'ABCE','ABCF') THEN (grp = 'A' AND rnk = 3)
                                      WHEN (SELECT concat_ws('', collect_list(grp))
                                            FROM cte5
                                            WHERE rnk = 3) IN ('ABDE', 'ABDF') THEN (grp = 'D' AND rnk = 3)
                                      WHEN (SELECT concat_ws('', collect_list(grp))
                                            FROM cte5
                                            WHERE rnk = 3) IN ('ABEF', 'ACDE','ACEF','ADEF','BCDE') THEN (grp = 'E' AND rnk = 3)
                                      WHEN (SELECT concat_ws('', collect_list(grp))
                                            FROM cte5
                                            WHERE rnk = 3) IN ('ACDF', 'BCDF','BCEF','BDEF','CDEF') THEN (grp = 'F' AND rnk = 3)
                                      END
                                UNION
                                SELECT name, grp, rnk FROM cte5 WHERE grp = 'A' AND rnk = 1
                                UNION
                                SELECT name, grp, rnk FROM cte5 WHERE grp = 'C' AND rnk = 2
                                UNION
                                SELECT name, grp, rnk FROM cte5 WHERE grp = 'F' AND rnk = 1
                                UNION
                                SELECT name, grp, rnk
                                FROM cte5
                                WHERE CASE 
                                      WHEN (SELECT concat_ws('', collect_list(grp))
                                            FROM cte5
                                            WHERE rnk = 3) IN ('ABEF','ACDE','ACDF','ACEF','ADEF') THEN (grp = 'A' AND rnk = 3)
                                      WHEN (SELECT concat_ws('', collect_list(grp))
                                            FROM cte5
                                            WHERE rnk = 3) IN ('ABDE','ABDF','BCDF','BCEF','BDEF') THEN (grp = 'B' AND rnk = 3)
                                      WHEN (SELECT concat_ws('', collect_list(grp))
                                            FROM cte5
                                            WHERE rnk = 3) IN ('ABCD','ABCE','ABCF','BCDE','CDEF') THEN (grp = 'C' AND rnk = 3)
                                      END
                                UNION
                                SELECT name, grp, rnk FROM cte5 WHERE grp = 'D' AND rnk = 2
                                UNION
                                SELECT name, grp, rnk FROM cte5 WHERE grp = 'E' AND rnk = 2
                                UNION
                                SELECT name, grp, rnk FROM cte5 WHERE grp = 'E' AND rnk = 1
                                UNION 
                                SELECT name, grp, rnk
                                FROM cte5
                                WHERE CASE 
                                      WHEN (SELECT concat_ws('', collect_list(grp))
                                            FROM cte5
                                            WHERE rnk = 3) IN ('ABDE', 'ABDF') THEN (grp = 'A' AND rnk = 3)
                                      WHEN (SELECT concat_ws('', collect_list(grp))
                                            FROM cte5
                                            WHERE rnk = 3) IN ('ABCD', 'ABCE','ABCF','ABEF','BCDE') THEN (grp = 'B' AND rnk = 3)
                                      WHEN (SELECT concat_ws('', collect_list(grp))
                                            FROM cte5
                                            WHERE rnk = 3) IN ('ACDE', 'ACDF','ACEF','BCDF','BCEF') THEN (grp = 'C' AND rnk = 3)
                                      WHEN (SELECT concat_ws('', collect_list(grp))
                                            FROM cte5
                                            WHERE rnk = 3) IN ('ADEF', 'BDEF','CDEF') THEN (grp = 'D' AND rnk = 3)
                                      END
                                UNION 
                                SELECT name, grp, rnk

 FROM cte5 WHERE grp = 'D' AND rnk = 1
                                UNION 
                                SELECT name, grp, rnk FROM cte5 WHERE grp = 'F' AND rnk = 2
                                UNION 
                                SELECT name, grp, rnk FROM cte5 WHERE grp = 'C' AND rnk = 1
                                UNION 
                                SELECT name, grp, rnk
                                FROM cte5
                                WHERE CASE 
                                      WHEN (SELECT concat_ws('', collect_list(grp))
                                            FROM cte5
                                            WHERE rnk = 3) IN ('ABCD','ACDE','ACDF','BCDE','BCDF') THEN (grp = 'D' AND rnk = 3)
                                      WHEN (SELECT concat_ws('', collect_list(grp))
                                            FROM cte5
                                            WHERE rnk = 3) IN ('ABCE','ABDE','BCEF','BDEF','CDEF') THEN (grp = 'E' AND rnk = 3)
                                      WHEN (SELECT concat_ws('', collect_list(grp))
                                            FROM cte5
                                            WHERE rnk = 3) IN ('ABCF','ABDF','ABEF','ACEF','ADEF') THEN (grp = 'F' AND rnk = 3)
                                      END
                                UNION 
                                SELECT name, grp, rnk FROM cte5 WHERE grp = 'A' AND rnk = 2
                                UNION 
                                SELECT name, grp, rnk FROM cte5 WHERE grp = 'B' AND rnk = 2
                            )
                            SELECT *, NTILE (8) OVER (ORDER BY (SELECT 0)) AS matches FROM cte6
                        )
                        SELECT 
                            CONCAT(a.name," ",a.grp,a.rnk,"    VS    ",b.name," ",b.grp,b.rnk) AS round_16_fixtures,
                            a.matches, 
                            CASE 
                                WHEN ROUND(RAND()*2) = 0 THEN "WIN" 
                                ELSE "LOSE" 
                            END AS team_1, 
                            CASE 
                                WHEN team_1 = "WIN" THEN "LOSE" 
                                ELSE "WIN" 
                            END AS team_2,
                            CASE 
                                WHEN team_1 = "WIN" THEN CONCAT(a.name," ",a.grp,a.rnk) 
                                ELSE CONCAT(b.name," ",b.grp,b.rnk) 
                            END AS quarter_finalist 
                        FROM 
                            cte7 a 
                        INNER JOIN 
                            cte7 b ON a.matches = b.matches AND a.name > b.name
                    )
                    SELECT 
                        quarter_finalist, NTILE(4) OVER (ORDER BY (SELECT 0)) AS matches 
                    FROM 
                        cte8
                )
                SELECT 
                    CONCAT(a.quarter_finalist,"   VS   ",b.quarter_finalist) AS quarter_final_fixtures,
                    a.matches,
                    CASE 
                        WHEN ROUND(RAND()*2) = 0 THEN "WIN" 
                        ELSE "LOSE" 
                    END AS team_1, 
                    CASE 
                        WHEN team_1 = "WIN" THEN "LOSE" 
                        ELSE "WIN" 
                    END AS team_2,
                    CASE 
                        WHEN team_1 = "WIN" THEN a.quarter_finalist 
                        ELSE b.quarter_finalist 
                    END AS semi_finalist 
                FROM 
                    cte9 a 
                INNER JOIN 
                    cte9 b ON a.matches = b.matches AND a.quarter_finalist > b.quarter_finalist 
                ORDER BY 
                    a.matches
            )
            SELECT 
                semi_finalist, NTILE(2) OVER (ORDER BY (SELECT 0)) AS matches 
            FROM 
                cte10
        )
        SELECT 
            CONCAT(a.semi_finalist,"   VS   ",b.semi_finalist) AS semi_final_fixtures,
            a.matches,
            CASE 
                WHEN ROUND(RAND()*2) = 0 THEN "WIN" 
                ELSE "LOSE" 
            END AS team_1, 
            CASE 
                WHEN team_1 = "WIN" THEN "LOSE" 
                ELSE "WIN" 
            END AS team_2,
            CASE 
                WHEN team_1 = "WIN" THEN a.semi_finalist 
                ELSE b.semi_finalist 
            END AS finalist 
        FROM 
            cte11 a 
        INNER JOIN 
            cte11 b ON a.matches = b.matches AND a.semi_finalist > b.semi_finalist 
        ORDER BY 
            a.matches
    )
    SELECT 
        finalist, NTILE(1) OVER (ORDER BY (SELECT 0)) AS matches 
    FROM 
        cte12
)
SELECT 
    CONCAT(a.finalist,"   VS   ",b.finalist) AS final_match,
    a.matches,
    CASE 
        WHEN ROUND(RAND()*2) = 0 THEN "WIN" 
        ELSE "LOSE" 
    END AS team_1, 
    CASE 
        WHEN team_1 = "WIN" THEN "LOSE" 
        ELSE "WIN" 
    END AS team_2,
    CASE 
        WHEN team_1 = "WIN" THEN a.finalist 
        ELSE b.finalist 
    END AS Winner 
FROM 
    cte13 a 
INNER JOIN 
    cte13 b ON a.matches = b.matches AND a.finalist > b.finalist;


final_match,matches,team_1,team_2,Winner
England C2 VS Croatia B2,1,LOSE,WIN,Croatia B2


#####I JUST COMPLETED THIS TASK WITH THE HELP OF CREATING A SINGLE TABLE. 
#####IT WOULD BE MORE SOPHISTICATED IF WE CREATED MULTIPLE TABLES.
#####BUT OUR CONDITION IS TO COMPLETE IT IN MINIMUM NUMBER OF TABLES.
#####THAT'S WHY I CHOOSE THIS METHOD OF MULTIPLE CTE'S INSTEAD OF CREATING MULTIPLE TABLES. 